In [ ]:
#Import of useful librairies
import cv2
import math
import numpy as np
from skimage import measure
import statistics as stat
import time

In [ ]:
handoverIn="./Outputs/handpics/cows"
handover='./Outputs/handouts/cows'
it=200

In [ ]:
N=2

In [ ]:
frame_height=1080
frame_width=1920

In [ ]:
#import particles their weigths and the anchor at handover time
particles=np.load(handoverIn+"_particles_data2_"+str(it)+".npy")
weights=np.load(handoverIn+'_weights_data2_'+str(it)+'.npy')
est=np.load(handoverIn+"_estimation_data2_"+str(it)+".npy")

n_particles=len(particles)

In [ ]:
def draw_bb(frame,BB,x,y):
        """This function draw BB and center of object"""
        cv2.circle(frame,(int(x),int(y)),5,[0,255,0],3)
        cv2.rectangle(frame, (BB[0],BB[1]), (BB[0]+BB[2],BB[1]+BB[3]), (0,0,255), 2)
        return frame

In [ ]:
def return_bb_coordinates(anchor,height,width):
    """This function return the BB coordinates. The y coordinates of the top
    left corner and the right bottom corner are first returned, while the x coordinates 
    come last."""

    dy1=int(anchor[1]-height/2)
    if(dy1<0):dy1=0

    dy2=int(anchor[1]+height/2)
    if (dy2>frame_height):dy2=frame_height

    dx1=int(anchor[0]-width/2)
    if (dx1<0): dx1=0

    dx2=int(anchor[0]+width/2)
    if (dx2>frame_width):dx2=frame_width

    return(dy1,dy2,dx1,dx2)


In [ ]:
def max_min_ssim(particles,ref,fov,height,width,threshold=0.9):
    """This function returns the maximal ssim index and the corresponding anchor 
    coordinates given an array of particles""" 
    
    ssims=[]
    for i in range (len(particles)):
        BB=return_bb_coordinates(particles[i],height,width)
        roi=fov[BB[0]:BB[1],BB[2]:BB[3]]
        
        ssims.append(measure.compare_ssim(ref,roi,multichannel=True,\
                                      gaussian_weights=True))
        
        #ssims.append(measure.compare_ssim(ref,roi,multichannel=True,\
        #                             win_size=3))
    
    max_ssim_index=ssims.index(max(ssims))
    min_ssim_index=ssims.index(min(ssims))
    
    return particles[max_ssim_index],ssims[max_ssim_index],max_ssim_index,\
            particles[min_ssim_index],ssims[min_ssim_index],min_ssim_index

In [ ]:
fov_A=cv2.imread(handoverIn+'_HandoverA_data2_'+str(it)+'.png')

In [ ]:
#tx and ty are the amount from which the FOVs are translated from one drone to another.
# It was assumed this amount is known either empirically or with deduction through drones
#Characteristics. We need to apply noise to it as it should be an uncertain measure.

std_fov=6
std_sampling=8

#translation in x and y directions with noises
tx0,ty0=-200,10
tx=tx0+std_fov*np.random.randn()
ty=ty0+std_fov*np.random.randn()

#Height and width of BB around target animal
height, width, channels = 224,320,3

#Simulation of FOVb given anchorA. We assume that the FOV are translated by tx and ty
#with noise.

num_rows, num_cols = fov_A.shape[:2]
translation_matrix = np.float32([[1,0,tx],[0,1,ty]])
fov_B=cv2.warpAffine(fov_A,translation_matrix,(num_cols, num_rows))
cv2.imwrite(handoverIn+'_HandoverB_data2_'+str(it)+'.png',fov_B)


## First method

In [ ]:
start_time = time.time()

In [ ]:
#anchorA transmitted
anchor_A=list(est)

#List containnig the ssims for the first method
ssim_anchorB_l=[]

ssim=0

for i in range(N):
    
    #anchorB given noisy particles transmitted with their weights
    anchor_B=[0,0]
    noisy_particles=np.zeros((n_particles,2))
    noisy_particles[:,0]=tx0+particles[:, 0]+std_fov*np.random.randn(n_particles)
    noisy_particles[:,1]=ty0+particles[:, 1]+std_fov*np.random.randn(n_particles)

    anchor_B[0] = np.average(noisy_particles[:,0], weights=weights, axis=0).astype(int)
    anchor_B[1] = np.average(noisy_particles[:,1], weights=weights, axis=0).astype(int)
    
    #return droneA ROI
    BBa=return_bb_coordinates(anchor_A,height,width)
    roi_A=fov_A[BBa[0]:BBa[1],BBa[2]:BBa[3]]

    #return droneB ROI
    BBb=return_bb_coordinates(anchor_B,height,width)
    roi_B=fov_B[BBb[0]:BBb[1],BBb[2]:BBb[3]]
    
    #Draw droneA and droneB ROI and FOV for one iteration
    if i==1:
        draw_A=draw_bb(fov_A.copy(),(BBa[2],BBa[0],width,height),anchor_A[0],anchor_A[1])
        cv2.imwrite(handover+'_ff_white_BB_A1.jpg',draw_A)

        draw_B=draw_bb(fov_B.copy(),(BBb[2],BBb[0],width,height),anchor_B[0],anchor_B[1])
        cv2.imwrite(handover+'_ff_white_BB_B1.jpg',draw_B)
    
    #Compute the SSIM measure given two roi with the parameter gaussian_weights activate
    ssim=measure.compare_ssim(roi_A,roi_B, multichannel=True,\
                                          gaussian_weights=True)
    
    ssim_anchorB_l.append(ssim)
    

In [ ]:
prog_duration= time.time() - start_time
prog_duration

In [ ]:
prog_duration/N

In [ ]:
prog_duration/60

In [ ]:
prog_duration/(60*N)

In [ ]:
stat.mean(ssim_anchorB_l),stat.stdev(ssim_anchorB_l),\
max(ssim_anchorB_l),min(ssim_anchorB_l)

## Second Method

In [ ]:
start_time = time.time()

In [ ]:
#anchorA transmitted
anchor_A=list(est)

#List containnig the ssims for the second method
ssim_noisy_particles_l=[]

ssim_max=0

for i in range(N):
    
    #Compute the noisy transmitted particles and use them to find and draw the combo
    #that gives the highest ssim 
    noisy_particles=np.zeros((n_particles,2))
    noisy_particles[:,0]=tx0+particles[:, 0]+std_fov*np.random.randn(n_particles)
    noisy_particles[:,1]=ty0+particles[:, 1]+std_fov*np.random.randn(n_particles)

    #return droneA ROI
    BBa=return_bb_coordinates(anchor_A,height,width)
    roi_A=fov_A[BBa[0]:BBa[1],BBa[2]:BBa[3]]
    
    #Compute the ssim with all noisy particles,retrieve the one that gives the maximal ssim
    anchor_ssim=max_min_ssim(noisy_particles,roi_A,fov_B,height,width)
    anchor_max,ssim_max=list(anchor_ssim[0]),anchor_ssim[1]
     
    #Draw droneA and droneB ROI and FOV for one iteration
    if i==1:
        draw_A=draw_bb(fov_A.copy(),(BBa[2],BBa[0],width,height),anchor_A[0],anchor_A[1])
        cv2.imwrite(handover+'_ff_white_BB_A2.jpg',draw_A)

        BB_max=return_bb_coordinates(anchor_max,height,width)
        draw_max=draw_bb(fov_B.copy(),(BB_max[2],BB_max[0],width,height),anchor_max[0],\
                         anchor_max[1])
        cv2.imwrite(handover+'_ff_white_BB_B2.jpg',draw_max)

    ssim_noisy_particles_l.append(ssim_max)
    

In [ ]:
prog_duration= time.time() - start_time
prog_duration

In [ ]:
prog_duration/N

In [ ]:
prog_duration/60

In [ ]:
prog_duration/(60*N)

In [ ]:
stat.mean(ssim_noisy_particles_l),stat.stdev(ssim_noisy_particles_l),\
max(ssim_noisy_particles_l),min(ssim_noisy_particles_l)

## Third Method

### New Particles = 2000

In [ ]:
n_particles_sampled=2000
start_time = time.time()

In [ ]:
#anchorA transmitted
anchor_A=list(est)

#List containnig the ssims for the second method
ssim_new_particles_l=[]

ssim_max=0

for i in range(N):
    
    #Compute the noisy transmitted particles and use them to find and draw the combo
    #that gives the highest ssim 
    anchor_B=[0,0]
    noisy_particles=np.zeros((n_particles,2))
    noisy_particles[:,0]=tx0+particles[:, 0]+std_fov*np.random.randn(n_particles)
    noisy_particles[:,1]=ty0+particles[:, 1]+std_fov*np.random.randn(n_particles)

    anchor_B[0] = np.average(noisy_particles[:,0], weights=weights, axis=0).astype(int)
    anchor_B[1] = np.average(noisy_particles[:,1], weights=weights, axis=0).astype(int)
    

    #return droneA ROI
    BBa=return_bb_coordinates(anchor_A,height,width)
    roi_A=fov_A[BBa[0]:BBa[1],BBa[2]:BBa[3]]
    
    #Use new sampled particles around transmitted anchor and find and draw the combo that gives
    #the highest ssim
    particles_high=np.zeros((n_particles_sampled,2))
    particles_high[:,0]=anchor_B[0]+std_sampling*np.random.randn(n_particles_sampled)
    particles_high[:,1]=anchor_B[1]+std_sampling*np.random.randn(n_particles_sampled)

    anchor_ssim=max_min_ssim(particles_high,roi_A,fov_B,height,width)
    anchor_max,ssim_max=list(anchor_ssim[0]),anchor_ssim[1]
    
    #Draw droneA and droneB ROI and FOV for one iteration
    if i==1:
        draw_A=draw_bb(fov_A.copy(),(BBa[2],BBa[0],width,height),anchor_A[0],anchor_A[1])
        cv2.imwrite(handover+'_ff_white_BB_A3_'+str(n_particles_sampled)+'.jpg',draw_A)

        BB_max=return_bb_coordinates(anchor_max,height,width)
        draw_max=draw_bb(fov_B.copy(),(BB_max[2],BB_max[0],width,height),anchor_max[0],\
                         anchor_max[1])
        cv2.imwrite(handover+'_ff_white_BB_B3_'+str(n_particles_sampled)+'.jpg',draw_max)
    
    ssim_new_particles_l.append(ssim_max)
        

In [ ]:
prog_duration= time.time() - start_time
prog_duration

In [ ]:
prog_duration/N

In [ ]:
prog_duration/60

In [ ]:
prog_duration/(60*N)

In [ ]:
stat.mean(ssim_new_particles_l),stat.stdev(ssim_new_particles_l),\
max(ssim_new_particles_l),min(ssim_new_particles_l)

### New Particles = 4000

In [ ]:
n_particles_sampled=4000
start_time = time.time()

In [ ]:
#anchorA transmitted
anchor_A=list(est)

#List containnig the ssims for the second method
ssim_new_particles_l=[]

ssim_max=0

for i in range(N):
    
    #Compute the noisy transmitted particles and use them to find and draw the combo
    #that gives the highest ssim 
    anchor_B=[0,0]
    noisy_particles=np.zeros((n_particles,2))
    noisy_particles[:,0]=tx0+particles[:, 0]+std_fov*np.random.randn(n_particles)
    noisy_particles[:,1]=ty0+particles[:, 1]+std_fov*np.random.randn(n_particles)

    anchor_B[0] = np.average(noisy_particles[:,0], weights=weights, axis=0).astype(int)
    anchor_B[1] = np.average(noisy_particles[:,1], weights=weights, axis=0).astype(int)
    

    #return droneA ROI
    BBa=return_bb_coordinates(anchor_A,height,width)
    roi_A=fov_A[BBa[0]:BBa[1],BBa[2]:BBa[3]]
    
    #Use new sampled particles around transmitted anchor and find and draw the combo that gives
    #the highest ssim
    particles_high=np.zeros((n_particles_sampled,2))
    particles_high[:,0]=anchor_B[0]+std_sampling*np.random.randn(n_particles_sampled)
    particles_high[:,1]=anchor_B[1]+std_sampling*np.random.randn(n_particles_sampled)

    anchor_ssim=max_min_ssim(particles_high,roi_A,fov_B,height,width)
    anchor_max,ssim_max=list(anchor_ssim[0]),anchor_ssim[1]
    
    #Draw droneA and droneB ROI and FOV for one iteration
    if i==1:
        draw_A=draw_bb(fov_A.copy(),(BBa[2],BBa[0],width,height),anchor_A[0],anchor_A[1])
        cv2.imwrite(handover+'_ff_white_BB_A3_'+str(n_particles_sampled)+'.jpg',draw_A)

        BB_max=return_bb_coordinates(anchor_max,height,width)
        draw_max=draw_bb(fov_B.copy(),(BB_max[2],BB_max[0],width,height),anchor_max[0],\
                         anchor_max[1])
        cv2.imwrite(handover+'_ff_white_BB_B3_'+str(n_particles_sampled)+'.jpg',draw_max)
    
    ssim_new_particles_l.append(ssim_max)
        

In [ ]:
prog_duration= time.time() - start_time
prog_duration

In [ ]:
prog_duration/N

In [ ]:
prog_duration/60

In [ ]:
prog_duration/(60*N)

In [ ]:
stat.mean(ssim_new_particles_l),stat.stdev(ssim_new_particles_l),\
max(ssim_new_particles_l),min(ssim_new_particles_l)

### New Particles = 6000 

In [ ]:
n_particles_sampled=6000
start_time = time.time()

In [ ]:
#anchorA transmitted
anchor_A=list(est)

#List containnig the ssims for the second method
ssim_new_particles_l=[]

ssim_max=0

for i in range(N):
    
    #Compute the noisy transmitted particles and use them to find and draw the combo
    #that gives the highest ssim 
    anchor_B=[0,0]
    noisy_particles=np.zeros((n_particles,2))
    noisy_particles[:,0]=tx0+particles[:, 0]+std_fov*np.random.randn(n_particles)
    noisy_particles[:,1]=ty0+particles[:, 1]+std_fov*np.random.randn(n_particles)

    anchor_B[0] = np.average(noisy_particles[:,0], weights=weights, axis=0).astype(int)
    anchor_B[1] = np.average(noisy_particles[:,1], weights=weights, axis=0).astype(int)
    

    #return droneA ROI
    BBa=return_bb_coordinates(anchor_A,height,width)
    roi_A=fov_A[BBa[0]:BBa[1],BBa[2]:BBa[3]]
    
    #Use new sampled particles around transmitted anchor and find and draw the combo that gives
    #the highest ssim
    particles_high=np.zeros((n_particles_sampled,2))
    particles_high[:,0]=anchor_B[0]+std_sampling*np.random.randn(n_particles_sampled)
    particles_high[:,1]=anchor_B[1]+std_sampling*np.random.randn(n_particles_sampled)

    anchor_ssim=max_min_ssim(particles_high,roi_A,fov_B,height,width)
    anchor_max,ssim_max=list(anchor_ssim[0]),anchor_ssim[1]
    
    #Draw droneA and droneB ROI and FOV for one iteration
    if i==1:
        draw_A=draw_bb(fov_A.copy(),(BBa[2],BBa[0],width,height),anchor_A[0],anchor_A[1])
        cv2.imwrite(handover+'_ff_white_BB_A3_'+str(n_particles_sampled)+'.jpg',draw_A)

        BB_max=return_bb_coordinates(anchor_max,height,width)
        draw_max=draw_bb(fov_B.copy(),(BB_max[2],BB_max[0],width,height),anchor_max[0],\
                         anchor_max[1])
        cv2.imwrite(handover+'_ff_white_BB_B3_'+str(n_particles_sampled)+'.jpg',draw_max)
    
    ssim_new_particles_l.append(ssim_max)
        

In [ ]:
prog_duration= time.time() - start_time
prog_duration

In [ ]:
prog_duration/N

In [ ]:
prog_duration/60

In [ ]:
prog_duration/(60*N)

In [ ]:
stat.mean(ssim_new_particles_l),stat.stdev(ssim_new_particles_l),\
max(ssim_new_particles_l),min(ssim_new_particles_l)

### New Particles = 8000 

In [ ]:
n_particles_sampled=8000
start_time = time.time()

In [ ]:
#anchorA transmitted
anchor_A=list(est)

#List containnig the ssims for the second method
ssim_new_particles_l=[]

ssim_max=0

for i in range(N):
    
    #Compute the noisy transmitted particles and use them to find and draw the combo
    #that gives the highest ssim 
    anchor_B=[0,0]
    noisy_particles=np.zeros((n_particles,2))
    noisy_particles[:,0]=tx0+particles[:, 0]+std_fov*np.random.randn(n_particles)
    noisy_particles[:,1]=ty0+particles[:, 1]+std_fov*np.random.randn(n_particles)

    anchor_B[0] = np.average(noisy_particles[:,0], weights=weights, axis=0).astype(int)
    anchor_B[1] = np.average(noisy_particles[:,1], weights=weights, axis=0).astype(int)
    

    #return droneA ROI
    BBa=return_bb_coordinates(anchor_A,height,width)
    roi_A=fov_A[BBa[0]:BBa[1],BBa[2]:BBa[3]]
    
    #Use new sampled particles around transmitted anchor and find and draw the combo that gives
    #the highest ssim
    particles_high=np.zeros((n_particles_sampled,2))
    particles_high[:,0]=anchor_B[0]+std_sampling*np.random.randn(n_particles_sampled)
    particles_high[:,1]=anchor_B[1]+std_sampling*np.random.randn(n_particles_sampled)

    anchor_ssim=max_min_ssim(particles_high,roi_A,fov_B,height,width)
    anchor_max,ssim_max=list(anchor_ssim[0]),anchor_ssim[1]
    
    #Draw droneA and droneB ROI and FOV for one iteration
    if i==1:
        draw_A=draw_bb(fov_A.copy(),(BBa[2],BBa[0],width,height),anchor_A[0],anchor_A[1])
        cv2.imwrite(handover+'_ff_white_BB_A3_'+str(n_particles_sampled)+'.jpg',draw_A)

        BB_max=return_bb_coordinates(anchor_max,height,width)
        draw_max=draw_bb(fov_B.copy(),(BB_max[2],BB_max[0],width,height),anchor_max[0],\
                         anchor_max[1])
        cv2.imwrite(handover+'_ff_white_BB_B3_'+str(n_particles_sampled)+'.jpg',draw_max)
    
    ssim_new_particles_l.append(ssim_max)
        

In [ ]:
prog_duration= time.time() - start_time
prog_duration

In [ ]:
prog_duration/N

In [ ]:
prog_duration/60

In [ ]:
prog_duration/(60*N)

In [ ]:
stat.mean(ssim_new_particles_l),stat.stdev(ssim_new_particles_l),\
max(ssim_new_particles_l),min(ssim_new_particles_l)